In [1]:
import time
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import csv
# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
#Keras
import keras
from keras import models
from keras import layers

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: F

In [47]:
path = '../data/kaggle-train-small/audio/'

In [73]:
start = time.time()

files = os.listdir(path)
stft = []
rmse = []
spec_cent = []
spec_bw = []
rolloff = []
zcr = []
mfcc = []

for f in files:
    filename = os.path.join(path, str(f))
    y, sr = librosa.load(filename, mono=True, duration=120)
    stft.append(librosa.feature.chroma_stft(y=y, sr=sr))
    rmse.append(librosa.feature.rms(y=y))
    spec_cent.append(librosa.feature.spectral_centroid(y=y, sr=sr))
    spec_bw.append(librosa.feature.spectral_bandwidth(y=y, sr=sr))
    rolloff.append(librosa.feature.spectral_rolloff(y=y, sr=sr))
    zcr.append(librosa.feature.zero_crossing_rate(y))
    mfcc.append(librosa.feature.mfcc(y=y, sr=sr))
    
stft = np.array(stft)
rmse = np.array(rmse)
spec_cent = np.array(spec_cent)
rolloff = np.array(rolloff)
zcr = np.array(zcr)
mfcc = np.array(mfcc)

df_audio = pd.DataFrame(files, columns=['filename'])
df_audio['audiotype'] = df_audio['filename'].str.extract(pat = '_([a-z]*)_')
df_audio['instrument'] = df_audio['filename'].str.extract(pat = '^([a-z]*)_')
df_audio['chroma_stft_mean'] = np.mean(stft, axis=(1,2))
df_audio['chroma_stft_std'] = np.std(stft, axis=(1,2))
df_audio['rmse_mean'] = np.mean(rmse, axis=(1,2))
df_audio['rmse_std'] = np.std(rmse, axis=(1,2))
df_audio['spec_cent_mean'] = np.mean(spec_cent, axis=(1,2))
df_audio['spec_cent_std'] = np.std(spec_cent, axis=(1,2))
df_audio['spec_bw_mean'] = np.mean(spec_bw, axis=(1,2))
df_audio['spec_bw_std'] = np.std(spec_bw, axis=(1,2))
df_audio['rolloff_mean'] = np.mean(rolloff, axis=(1,2))
df_audio['rolloff_std'] = np.std(rolloff, axis=(1,2))
df_audio['zcr_mean'] = np.mean(zcr, axis=(1,2))
df_audio['zcr_std'] = np.std(zcr, axis=(1,2))
for j in range(0, 20):
    df_audio[f' mfcc_mean{j}'] = np.mean(elem[j,:])
    df_audio[f' mfcc_std{j}'] = np.std(elem[j,:])    
    
print("Extracting the features took " + str((time.time()-start)/60) + "mins")

/usr/local/lib/python3.6/dist-packages/librosa/core/pitch.py:146: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


KeyboardInterrupt: 

In [49]:
df_audio.head()

,filename,audiotype,instrument,chroma_stft_mean,chroma_stft_std,rmse_mean,rmse_std,spec_cent_mean,spec_cent_std,spec_bw_mean,...,mfcc_mean15,mfcc_std15,mfcc_mean16,mfcc_std16,mfcc_mean17,mfcc_std17,mfcc_mean18,mfcc_std18,mfcc_mean19,mfcc_std19
0,keyboard_electronic_035-064-127.wav,electronic,keyboard,0.183051,0.298926,0.100272,0.132926,866.78674,950.658223,742.024435,...,-13.679363,5.580251,0.602789,4.130479,9.30153,5.73318,3.138919,6.195168,-17.187359,6.847628
1,reed_acoustic_014-063-075.wav,acoustic,reed,0.183051,0.298926,0.100272,0.132926,866.78674,950.658223,742.024435,...,-13.679363,5.580251,0.602789,4.130479,9.30153,5.73318,3.138919,6.195168,-17.187359,6.847628
2,organ_electronic_106-049-100.wav,electronic,organ,0.183051,0.298926,0.100272,0.132926,866.78674,950.658223,742.024435,...,-13.679363,5.580251,0.602789,4.130479,9.30153,5.73318,3.138919,6.195168,-17.187359,6.847628
3,keyboard_electronic_005-035-050.wav,electronic,keyboard,0.183051,0.298926,0.100272,0.132926,866.78674,950.658223,742.024435,...,-13.679363,5.580251,0.602789,4.130479,9.30153,5.73318,3.138919,6.195168,-17.187359,6.847628
4,bass_synthetic_041-064-075.wav,synthetic,bass,0.183051,0.298926,0.100272,0.132926,866.78674,950.658223,742.024435,...,-13.679363,5.580251,0.602789,4.130479,9.30153,5.73318,3.138919,6.195168,-17.187359,6.847628


In [6]:
df_audio.dropna(inplace=True)

In [16]:
data = df_audio[[c for c in df_audio.columns if c not in ['filename', 'audiotype']]]

In [17]:
# defining target
encoder = LabelEncoder()
y = encoder.fit_transform(data['instrument'])
print(y[0:5])

[4 7 6 4 0]


In [23]:
# normalizing
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:,3:], dtype = float))
print(X[0])

[-0.49563467 -0.1020057   0.09112291 -0.7655021   0.08391973 -0.66337414
  0.11327354 -0.65955368  0.04001448 -0.52526675  0.02216956 -0.41619166
 -0.16605724 -0.48783638 -0.40454303 -0.08302579 -1.04472945  0.35342482
 -0.32188311 -0.45387657 -1.39071151  0.23547061 -1.17962198 -0.16813327
 -0.9030258  -0.55214999 -0.40960861 -0.17303188 -0.10431632 -0.52398778
  1.09934773  0.89539408  0.54066654 -0.21817366 -1.27864473  0.72764207
 -0.98832286 -0.02721584 -1.04845198 -0.06609157 -1.77646849  1.29036995
 -1.8161079   0.62867492 -0.62738056 -0.36668915  0.51905679 -0.04422229
  1.25038684  0.21905623]


In [24]:
# spliting of dataset into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [25]:
# creating a model
model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
              
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=128)
                    
# calculate accuracy
test_loss, test_acc = model.evaluate(X_test,y_test)
print('test_acc: ',test_acc)

# predictions
predictions = model.predict(X_test)
np.argmax(predictions[0])

W0809 14:27:04.035025 140367796909888 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0809 14:27:04.078443 140367796909888 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0809 14:27:04.082442 140367796909888 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0809 14:27:04.178236 140367796909888 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0809 14:27:04.207391 140367796909888 deprecation_wrappe

Epoch 1/20
800/800 [==============================] - 0s 397us/step - loss: 2.1852 - acc: 0.1937
Epoch 2/20
800/800 [==============================] - 0s 23us/step - loss: 1.8918 - acc: 0.3200
Epoch 3/20
800/800 [==============================] - 0s 20us/step - loss: 1.7363 - acc: 0.3950
Epoch 4/20
800/800 [==============================] - 0s 35us/step - loss: 1.6029 - acc: 0.4288
Epoch 5/20
800/800 [==============================] - 0s 35us/step - loss: 1.4967 - acc: 0.4788
Epoch 6/20
800/800 [==============================] - 0s 53us/step - loss: 1.4003 - acc: 0.5300
Epoch 7/20
800/800 [==============================] - 0s 33us/step - loss: 1.3079 - acc: 0.5750
Epoch 8/20
800/800 [==============================] - 0s 33us/step - loss: 1.2153 - acc: 0.6062
Epoch 9/20
800/800 [==============================] - 0s 30us/step - loss: 1.1295 - acc: 0.6388
Epoch 10/20
800/800 [==============================] - 0s 35us/step - loss: 1.0435 - acc: 0.6800
Epoch 11/20
800/800 [=================

6